# 实验三：有约束的极值求解


## 实验目的

1. 通过排课问题理解非线性规划问题;
2. 熟悉有约束的极值求解方法；
3. 了解排课算法优化算法；
4. 熟悉list,dataFrame等数据结构
5. 养成高质量代码编写习惯。

## 实验要求：

1. 编码规范
2. 代码高效
3. 注释充分，程序可读性好
4. 程序无bug
5. 方法接口规范定义

## 实验内容

根据以下约束和排班原则进行排课算法设计
1. 同一个任课教师在一个教学时间段只能进行一门课程教学
2. 同一个班级在一个教学时间段只能进行一门课程教学
3. 同一个教室在一个教学时间段只能进行一门课程教学
4. 排课教室必须满足学生人数要求
5. 一个教师每天不能超过6节课
6. 每个学生班级每天不能超过8节课
7. 每个教学班同一门课程每天不能超过2节课


## 实验步骤

## 1.算法设计

数据结构
![数据结构](实验三数据结构.png)

算法流程
![算法流程](实验三算法流程.png)

### 2.数据与处理

对教学任务信息，教室信息，教师信息，教学班级信息进行处理，组织排课算法需要的各数据和排课结果表，并对相关变量进行初始化



定义需要的数据结构

In [1]:
# 上课时间列表
schoolTime = [["第一大节", ["第01节课", "08:30-09:15"], ["第02节课", "09:20-10:05"]], 
              ["第二大节", ["第03节课", "10:25-11:10"], ["第04节课", "11:15-12:00"]], 
              ["第三大节", ["第05节课", "14:00-14:45"], ["第06节课", "14:50-15:35"]], 
              ["第四大节", ["第07节课", "15:55-16:40"], ["第08节课", "16:45-17:30"]], 
              ["第五大节", ["第09节课", "18:30-19:15"], ["第010节课", "19:20-10:05"]]]
week = ['周一','周二','周三','周四','周五','周六','周日']
# 班级信息类
class ClassInfo:
    def __init__(self, name, stuNum):
        self.name = name
        self.stuNum = stuNum  
    def __str__(self):
        return self.name
# 教师类
class Teacher:
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return self.name
# 课程类
class Course:
    def __init__(self, id, name, weekTime, total, classInfo, teacher, weekOfStartStop):
        self.id = id
        self.name = name
        self.weekTime = weekTime
        self.total = total
        self.classInfo = classInfo
        self.teacher = teacher
        self.weekOfStartStop = weekOfStartStop
    def __str__(self):
        return str(self.name)+" "+str(self.weekOfStartStop)+"周"
# 教室类
class Classroom:
    def __init__(self, id, type, contain):
        self.id = id
        self.type = type
        self.contain = contain
    def __str__(self):
        return str(self.id)
# 排课安排类
class Arrange:
    def __init__(self, course, classroom, schoolTime):
        self.course = course
        self.classroom = classroom
        self.schoolTime = schoolTime
    def __str__(self):
        return str(self.course)+" "+str(self.classroom)

导入所需要的包

In [2]:
import pandas as pd
import numpy as np

从excel表中读取数据

In [3]:
# 开课任务数据
data1 = pd.read_excel("实验三数据.xlsx",0)
# 教室信息数据
data2 = pd.read_excel("实验三数据.xlsx",1)

数据封装

In [4]:
# 课程列表
courseList = []
# 教室列表
classroomList = []
# 排课安排列表，dataFrame,每周7天每天5节大课
arrangeList = [[],[],[],[],[],[],[],
              [],[],[],[],[],[],[],
              [],[],[],[],[],[],[],
              [],[],[],[],[],[],[],
              [],[],[],[],[],[],[]]
'''
班级信息的获取
ClassInfo(data1.loc[0][20]+"、"+data1.loc[0][20], data1.loc[0][19]+data1.loc[0][19])
教师信息的获取
Teacher(data1.loc[0][22])
课程信息的获取
Course(data1.loc[0][0], data1.loc[0][1], data1.loc[0][3], data1.loc[0][5], ClassInfo, Teacher, data1.loc[0][4])
教室信息的获取
Classroom(data2.loc[0][0], data2.loc[0][1], data2.loc[0][2])
'''
for i in range(0, len(data1),2):
    courseList.append(Course(data1.loc[i][0], data1.loc[i][1], data1.loc[i][3], data1.loc[i][5],\
                             ClassInfo(data1.loc[i][20]+"、"+data1.loc[i+1][20], data1.loc[i][19]+data1.loc[i+1][19]), \
                             Teacher(data1.loc[i][22]), data1.loc[0][4]))
for i in range(len(data2)):
    classroomList.append(Classroom(data2.loc[i][0], data2.loc[i][1], data2.loc[i][2]))

### 3.算法实现

按照最短教学时间原则进行排课

#### 判断是否可以排课

> 今天该班级是否安排过该课程

In [5]:
def func1(course, weekday):
    for i in range(weekday,35,7):
        for j in arrangeList[i]:
            if j.course.name == course.name:
                if j.course.classInfo.name == course.classInfo.name:
                    return False
    return True
    pass

> 该老师当前是否有课程

In [6]:
def func2(course, i):
    for j in arrangeList[i]:
        if j.course.teacher.name == course.teacher.name:
            return False
    return True
    pass

> 该班级当前是否有课程

In [7]:
def func3(course, i):
    for j in arrangeList[i]:
        if j.course.classInfo.name == course.classInfo.name:
            return False
    return True
    pass

> 该教室当前是否有课程(若当前教室的当前时间有课程，则换一个教室继续判断)

In [8]:
def func4(classroom, i):
    for j in arrangeList[i]:
        if j.classroom.id == classroom.id:
            return False
    return True
    pass

> 该教室座位是否足够

In [9]:
def func5(course, classroom):
    if course.classInfo.stuNum <= classroom.contain:
        return True
    else:
        return False
    pass

> 该老师今天课程是否超过六节

In [10]:
def func6(course, weekday):
    #记录老师今天的课程数
    num = 0
    
    for i in range(weekday,35,7):
        for j in arrangeList[i]:
            if j.course.teacher.name == course.teacher.name:
                num += 1
    if num >= 3:
        return False
    else:
        return True
    pass

> 该班级今天课程是否超过八节

In [11]:
def func7(course, weekday):
    #记录班级今天的课程数
    num = 0
    
    for i in range(weekday,35,7):
        for j in arrangeList[i]:
            if j.course.classInfo.name == course.classInfo.name:
                num += 1
    if num >= 4:
        return False
    else:
        return True
    pass

#### 安排课程

In [12]:
def arrangeCourse(course):
    for classroom in classroomList:
        for i in range(35):
            if func1(course, i % 7):
                if func2(course, i):
                    if func3(course, i):
                        if func4(classroom, i):
                            if func5(course, classroom):
                                if func6(course, i % 7):
                                    if func7(course, i % 7):
                                        arrangeList[i].append(Arrange(course, classroom, i+1))
                                        return

for course in courseList:
    for i in range(course.weekTime // 2):
        arrangeCourse(course)

In [13]:
for i in range(35):
    for arrange in arrangeList[i]:
        print(str(arrange))

Linux程序设计 01-16周 10201
面向对象程序设计 01-16周 10202
数据库原理 01-16周 10203
Linux程序设计 01-16周 10201
高等数学A+（2） 01-16周 10202
用户体验设计 01-16周 10203
Linux程序设计 01-16周 10201
高等数学A+（2） 01-16周 10202
用户体验设计 01-16周 10203
Linux程序设计 01-16周 10201
高等数学A+（2） 01-16周 10202
面向对象程序设计 01-16周 10203
IT新技术 01-16周 10201
高等数学A+（2） 01-16周 10202
面向对象程序设计 01-16周 10203
IT新技术 01-16周 10201
高等数学A+（2） 01-16周 10202
三维建模技术（1） 01-16周 10203
IT新技术 01-16周 10201
高等数学A+（2） 01-16周 10202
三维建模技术（1） 01-16周 10203
IT新技术 01-16周 10201
大学物理B 01-16周 10202
J2EE常用框架技术 01-16周 10201
大学物理B 01-16周 10202
J2EE常用框架技术 01-16周 10201
大学物理B 01-16周 10202
J2EE常用框架技术 01-16周 10201
大学物理B 01-16周 10202
J2EE常用框架技术 01-16周 10201
C语言课程设计 01-16周 10202
软件系统分析与设计技术 01-16周 10201
C语言课程设计 01-16周 10202
软件系统分析与设计技术 01-16周 10201
C语言课程设计 01-16周 10202
软件系统分析与设计技术 01-16周 10201
C语言课程设计 01-16周 10202
软件系统分析与设计技术 01-16周 10201
大学物理A（1） 01-16周 10202
移动开发技术 01-16周 10201
大学物理A（1） 01-16周 10202
移动开发技术 01-16周 10201
数字媒体美术基础 01-16周 10202
移动开发技术 01-16周 10201
数字媒体美术基础 01-16周 10202
移动开发技术 01-16周 10201

### 4.算法优化

本部分为选做：
1. 可以使用蚁群算法等对排课算法进行优化，得到更优的排课方案；
2. 合理设计算法模块，使程序支持多种排课原则，比如按最短教学时间优先
3. 课程任务分布均匀优先
4. 教师要求优先

In [14]:
#code

### 5.结果输出

1. 教师课表
2. 学生班级课表
3. 教室课表

df = pd.DataFrame(columns=('周一','周二','周三','周四','周五','周六','周日'))

#### 教师课表

In [30]:
def teacherTable(teacher):
    df = pd.DataFrame(columns=week)
    tmp = ["", "", "", "", "", "", ""]
    for i in range(35):
        for arrange in arrangeList[i]:
            if arrange.course.teacher.name == teacher.name:
                tmp[i%7] = str(arrange)+" "+ str(arrange.course.teacher)+" "+str(arrange.course.classInfo.name)+" "+str(arrange.classroom)
                break
        if i%7 == 6:
            df.loc[schoolTime[i//7][0]] = tmp
            tmp = ["", "", "", "", "", "", ""]
    return df

In [32]:
teacher = Teacher("黄荣兵")
df = teacherTable(teacher)
print(df)
url = "E:\学习\课程\大三下\机器学习\实验\实验三\\" +teacher.name + " - 教师课表.xlsx"
df.to_excel(url)

['Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-1、软件(本)17-2 10201', 'Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-1、软件(本)17-2 10201', 'Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-3、软件(本)17-4 10201', 'Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-3、软件(本)17-4 10201', '', '', '']
['', '', '', '', '', '', '']
['', '', '', '', '', '', '']
['', '', '', '', '', '', '']
['', '', '', '', '数据库原理 01-16周 10202 黄荣兵 软件(本)18-1、软件(本)18-2 10202', '数据库原理 01-16周 10202 黄荣兵 软件(本)18-1、软件(本)18-2 10202', '']
                                                     周一  \
第一大节  Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-1、软件(本)17-2...   
第二大节                                                      
第三大节                                                      
第四大节                                                      
第五大节                                                      

                                                     周二  \
第一大节  Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-1、软件(本)17-2...   
第二大节                                                      
第三大节               

#### 学生班级课表

In [43]:
def classTable(classInfo):
    df = pd.DataFrame(columns=week)
    tmp = ["", "", "", "", "", "", ""]
    for i in range(35):
        for arrange in arrangeList[i]:
            if arrange.course.classInfo.name == classInfo.name:
                tmp[i%7] = str(arrange)+" "+ str(arrange.course.teacher)+" "+str(arrange.course.classInfo.name)+" "+str(arrange.classroom)
                break
        if i%7 == 6:

            df.loc[schoolTime[i//7][0]] = tmp
            tmp = ["", "", "", "", "", "", ""]
    return df

In [48]:
classInfo = ClassInfo("软件(本)17-1、软件(本)17-2", 1)
df = classTable(classInfo)
print(df)
url = "E:\学习\课程\大三下\机器学习\实验\实验三\\" +classInfo.name + " - 班级课表.xlsx"
df.to_excel(url)

                                                     周一  \
第一大节  Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-1、软件(本)17-2...   
第二大节                                                      
第三大节                                                      
第四大节                                                      
第五大节                                                      

                                                     周二  \
第一大节  Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-1、软件(本)17-2...   
第二大节  J2EE常用框架技术 01-16周 10201 于曦 软件(本)17-1、软件(本)17-2...   
第三大节                                                      
第四大节                                                      
第五大节                                                      

                                                     周三  \
第一大节                                                      
第二大节  J2EE常用框架技术 01-16周 10201 于曦 软件(本)17-1、软件(本)17-2...   
第三大节  移动开发技术 01-16周 10201 苏长明 软件(本)17-1、软件(本)17-2 10201   
第四大节                                                  

#### 教室课表

In [45]:
def classroomTable(classInfo):
    df = pd.DataFrame(columns=week)
    tmp = ["", "", "", "", "", "", ""]
    for i in range(35):
        for arrange in arrangeList[i]:
            if arrange.classroom.id == classroom.id:
                tmp[i%7] = str(arrange)+" "+ str(arrange.course.teacher)+" "+str(arrange.course.classInfo.name)
                break
        if i%7 == 6:

            df.loc[schoolTime[i//7][0]] = tmp
            tmp = ["", "", "", "", "", "", ""]
    return df

In [51]:
classroom = Classroom(10201,"多媒体实验室", 70)
df = classroomTable(classInfo)
print(df)
url = "E:\学习\课程\大三下\机器学习\实验\实验三\\" +str(classroom.id) + " - 教室课表.xlsx"
df.to_excel(url)

                                                    周一  \
第一大节    Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-1、软件(本)17-2   
第二大节        IT新技术 01-16周 10201 胡德昆 软件(本)17-3、软件(本)17-4   
第三大节  软件系统分析与设计技术 01-16周 10201 赵卫东 软件(本)17-3、软件(本)17-4   
第四大节       软件工程经济学 01-16周 10201 李立 软件(本)17-3、软件(本)17-4   
第五大节    数字媒体综合案例制作 01-16周 10201 赵荣 数媒(本)17-1、数媒(本)17-2   

                                                    周二  \
第一大节    Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-1、软件(本)17-2   
第二大节    J2EE常用框架技术 01-16周 10201 于曦 软件(本)17-1、软件(本)17-2   
第三大节  软件系统分析与设计技术 01-16周 10201 赵卫东 软件(本)17-3、软件(本)17-4   
第四大节       数字摄影与摄像 01-16周 10201 朱然 数媒(本)17-1、数媒(本)17-2   
第五大节    数字媒体综合案例制作 01-16周 10201 赵荣 数媒(本)17-1、数媒(本)17-2   

                                                  周三  \
第一大节  Linux程序设计 01-16周 10201 黄荣兵 软件(本)17-3、软件(本)17-4   
第二大节  J2EE常用框架技术 01-16周 10201 于曦 软件(本)17-1、软件(本)17-2   
第三大节     移动开发技术 01-16周 10201 苏长明 软件(本)17-1、软件(本)17-2   
第四大节     数字摄影与摄像 01-16周 10201 朱然 数媒(本)17-1、数媒(本)17-2   
第五大节       IT新技术 01-16

## 实验总结

在拿道题目的时候感觉很懵，但是经过逐步分析，发现这次实验只是代码多了一些，逻辑上还是很清楚的。
但是在各方面的优化以及面向对象的思想还是不太行